In [ ]:
!pip install transformers
!pip install datasets
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 29.5 MB/s 
     |████████████████████████████████| 6.6 MB 53.2 MB/s 
     |████████████████████████████████| 101 kB 11.7 MB/s 
     |████████████████████████████████| 596 kB 59.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 32.2 MB/s 
     |████████████████████████████████| 212 kB 48.8 MB/s 
     |████████████████████████████████| 140 kB 73.7 MB/s 
     |████████████████████████████████| 1.1 MB 55.4 MB/s 
     |████████████████████████████████| 127 kB 78.5 MB/s 
     |████████████████████████████████| 144 kB 60.8 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 
     |███████████

In [ ]:
import transformers
from sklearn.model_selection import train_test_split
from datasets import Dataset
from ast import literal_eval
from datasets import load_metric, DatasetDict
from transformers import (
    AutoTokenizer,
    DataCollatorForTokenClassification,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
)
from smart_open import open
from transformers import AutoTokenizer

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
label_encoding_dict = {
            "O": 0,
            "B-PERS": 1,
            "I-PERS": 2,
        }

In [ ]:
training_tagset = ['O', 'B-PERS', 'I-PERS']

In [ ]:
language = "fr"
transformer_model = {"fr": "dbmdz/bert-base-french-europeana-cased",}
hipe_datasets = {"fr": ["all-fr.jsonl.gz"]}

In [ ]:
lines = []
for hd in hipe_datasets[language]:
    with open(hd) as fr:
        lines += fr.readlines()

annotations = []
counter = 0
for line in lines:
    line = line.replace("B-HIDDEN", "B-PERS")
    line = line.replace("I-HIDDEN", "I-PERS")
    line = line.replace("B-PUBLIC", "B-PERS")
    line = line.replace("I-PUBLIC", "I-PERS")
    line = literal_eval(line)
    line["id"] = str(counter)
    counter += 1
    annotations.append(line)

dict_hf = {'id': [],
           'tokens': [],
           'ner_tags': []}

for a in annotations:
    dict_hf["id"].append(a["id"])
    dict_hf["tokens"].append(a["tokens"])
    dict_hf["ner_tags"].append(a["ner_tags"])

dataset = Dataset.from_dict(dict_hf)
train_testvalid = dataset.train_test_split(test_size=0.2, random_state=42)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, random_state=42)
dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})

In [ ]:
print(dataset["train"])
print(dataset["validation"])
print(dataset["test"])

Dataset({
    features: ['id', 'tokens', 'ner_tags'],
    num_rows: 4283
})
Dataset({
    features: ['id', 'tokens', 'ner_tags'],
    num_rows: 535
})
Dataset({
    features: ['id', 'tokens', 'ner_tags'],
    num_rows: 536
})


In [ ]:
dataset["test"][0]

{'id': '597',
 'ner_tags': ['O',
  'O',
  'B-PERS',
  'I-PERS',
  'I-PERS',
  'I-PERS',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 'tokens': ['—',
  '(',
  'C',
  '.',
  'P',
  '.',
  ')',
  '—',
  'Six',
  'nouveaux',
  'conseillers',
  'ont',
  'prêté',
  'serment',
  'lors',
  'de',
  'la',
  'dernière',
  'séance',
  ':',
  'MM',
  '.']}

In [ ]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = transformer_model[language]
batch_size = 16

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(training_tagset))

Downloading:   0%|          | 0.00/83.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/222k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-base-french-europeana-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not

In [ ]:
example = dataset["train"][4]
print(example["tokens"])
print(example["ner_tags"])

['Les', 'dix', 'lecteurs', 'de', 'REGARDS', 'qui', 'nous', 'adresseront', ':', 'a', ')', 'la', 'solution', 'juste', 'du', 'problème', ';', 'b', ')', 'qui', 'indiqueront', 'comme', 'nombre', 'de', 'ré¬', 'ponses', 'justes', 'reçues', 'par', 'REGARDS', 'les', 'chif¬', 'fres', 'se', 'rapprochant', 'le', 'plus', 'du', 'nombre', 'exact', 'recevront', 'gratuitement', 'un', 'livre', 'à', 'choisir', 'dans', 'la', 'liste', 'ci', '-', 'dessous', ':', 'HYDROCENTRALE', ',', 'roman', 'de', 'M', '.', 'Chagui¬', 'nian', ';', 'TERRES', 'DEFRICHEES', ',', 'roman', 'de', 'Cholo¬', 'khov', ';', 'LE', 'TALON', 'DE', 'FER', ',', 'roman', 'de', 'Jack', 'Lon¬', 'don', ';', 'HISTOIRE', 'DU', 'MOUVEMENT', 'OUVRIER', 'EN', 'FRANCE', ',', 'par', 'Vidal', ';', 'SOUVENIRS', 'D', "'", 'UN', 'BOLCHEVIK', ',', 'par', 'Piat¬', 'nitski', ';', 'DIX', 'JOURS', 'QUI', 'EBRANLERENT', 'LE', 'MON¬', 'DE', ',', 'par', 'John', 'Reed', ';', 'EUX', 'ET', 'NOUS', ',', 'par', 'Maxim', 'Gorki', '.']
['O', 'O', 'O', 'O', 'O', 'O', '

In [ ]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
#print(tokens)

In [ ]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(121, 161)

In [ ]:
#print(tokenized_input.word_ids())

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 11, 11, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 27, 28, 29, 30, 30, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 39, 40, 41, 42, 43, None]


In [ ]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

53 53


In [ ]:
label_all_tokens = True

In [ ]:
def tokenize_and_align_labels(examples):
    print(examples)
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label_encoding_dict[label[word_idx]])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label_encoding_dict[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
# tokenize_and_align_labels(dataset['train'][:5])

{'id': ['4635', '2099', '4780', '3043', '3126'], 'tokens': [[',', 'faci¬', 'lités', '.', 'Lebœuf', ',', '24', ',', 'av', '.', 'Wagram', '.', 'Wag', '.', '30', '-', '24', '.'], ['G', '.', 'Ribemont', '-', 'Dessaignes', '.'], ['640', 'Eugène', 'MERLE', ',', 'Directeur', 'Quotidien', 'Un', 'ministère', 'provisoire', '.', '.', '.', 'EN', 'ATTENDANT', 'LA', 'DÉCISION', 'DES', 'SOCIALISTES', '.', '.', '.'], ['200', 'ont', 'rejoint', 'Le', 'Soulier', 'de', 'verre', ',', 'dont', 'Georges', 'Neveu', 'a', 'écrit', 'le', 'scénario', 'et', 'les', 'dia¬', 'logues', '.', 'Le', 'rôle', 'principal', 'sera', 'tenu', 'par', 'Odette', 'Joyeux', ',', 'qui', 'incarnera', 'une', 'petite', 'fille', 'boiteuse', 'éperdue', 'de', 'danse', ';', 'c', "'", 'est', 'un', 'peu', ',', 'si', 'vous', 'vou¬', 'lez', ',', 'une', 'transposition', 'de', 'Cendril¬', 'lon', 'où', 'la', 'petite', 'fille', 'finit', 'par', 'deve¬', 'nir', 'une', 'grande', 'actrice', ',', 'pourtant', 'ce', 'n', "'", 'est', 'pas', 'un', 'film', 's

{'input_ids': [[2, 16, 1, 14634, 18, 7750, 27688, 16, 1288, 16, 392, 18, 17107, 18, 9133, 18, 770, 17, 1288, 18, 3], [2, 43, 18, 8746, 773, 384, 17, 14060, 6621, 212, 18, 3], [2, 13656, 5873, 8665, 912, 16, 9143, 28646, 4937, 694, 1983, 5465, 18, 18, 18, 2242, 16586, 10916, 2225, 1129, 25757, 706, 3712, 1861, 27636, 15307, 18, 18, 18, 3], [2, 2059, 507, 15598, 447, 20941, 434, 336, 7965, 16, 669, 2560, 1657, 9636, 68, 2087, 354, 30947, 12068, 353, 361, 1, 3159, 803, 18, 447, 2683, 5432, 866, 3071, 383, 22233, 15723, 640, 16, 405, 21816, 13543, 408, 2096, 1794, 9752, 619, 29952, 987, 204, 336, 8204, 31, 70, 11, 401, 390, 554, 16, 485, 578, 1, 24468, 16, 408, 1141, 16215, 336, 1, 4316, 675, 348, 2096, 1794, 8084, 383, 1, 6492, 408, 1079, 28911, 16, 4070, 463, 81, 11, 401, 435, 390, 7785, 441, 354, 2420, 919, 441, 348, 8204, 11848, 31, 399, 401, 521, 71, 11, 17910, 20070, 353, 1744, 422, 348, 9447, 367, 336, 348, 1287, 130, 348, 1, 6511, 11019, 18, 109, 39, 11, 401, 617, 819, 354, 1762, 4

In [ ]:
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

Parameter 'function'=<function tokenize_and_align_labels at 0x7fc18644b9e0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/5 [00:00<?, ?ba/s]

{'id': ['4635', '2099', '4780', '3043', '3126', '2547', '3776', '1315', '4701', '850', '978', '3208', '4603', '2544', '3389', '4074', '5347', '5200', '5240', '1613', '3671', '167', '3415', '1135', '348', '263', '4940', '4770', '1506', '3771', '423', '3759', '677', '1337', '4070', '1422', '4161', '2819', '1304', '4444', '1643', '890', '2221', '786', '951', '4787', '228', '2949', '4721', '1717', '2263', '619', '1143', '3151', '2880', '118', '2685', '961', '48', '738', '4795', '1294', '4821', '3892', '848', '1799', '1822', '4395', '2945', '4166', '3796', '748', '2424', '4008', '3619', '1742', '1919', '3939', '2326', '4139', '3697', '1921', '4946', '3013', '5008', '5164', '1982', '2125', '1913', '2226', '4083', '3660', '614', '4038', '4634', '827', '3760', '2492', '2078', '4864', '3851', '5060', '986', '3820', '4215', '1485', '1390', '2468', '50', '3107', '5129', '4394', '4256', '174', '467', '1718', '641', '1606', '1446', '1638', '717', '910', '404', '1370', '1704', '3213', '3809', '3799'

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"person-finetuned",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [ ]:
metric = load_metric("seqeval")

In [ ]:
# labels = [label_encoding_dict[i] for i in example[f"{task}_tags"]]
labels = [i for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'PERS': {'f1': 1.0, 'number': 7, 'precision': 1.0, 'recall': 1.0},
 'overall_accuracy': 1.0,
 'overall_f1': 1.0,
 'overall_precision': 1.0,
 'overall_recall': 1.0}

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [training_tagset[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [training_tagset[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: id, ner_tags, tokens. If id, ner_tags, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 4283
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2680


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.090978,0.738946,0.758706,0.748696,0.969880
2,0.116100,0.079900,0.761006,0.827736,0.792970,0.974197
3,0.116100,0.076532,0.808405,0.813433,0.810911,0.977175
4,0.037900,0.081723,0.807178,0.825249,0.816113,0.978581
5,0.037900,0.085413,0.802704,0.812189,0.807419,0.977276
6,0.020700,0.088840,0.806006,0.834577,0.820043,0.978681
7,0.020700,0.097868,0.812576,0.827736,0.820086,0.978246
8,0.011900,0.100273,0.824216,0.833955,0.829057,0.978849
9,0.011900,0.097994,0.822276,0.840174,0.831129,0.979552
10,0.008500,0.101640,0.818405,0.829602,0.823965,0.978614


The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: id, ner_tags, tokens. If id, ner_tags, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 535
  Batch size = 16
Saving model checkpoint to person-finetuned/checkpoint-268
Configuration saved in person-finetuned/checkpoint-268/config.json
Model weights saved in person-finetuned/checkpoint-268/pytorch_model.bin
tokenizer config file saved in person-finetuned/checkpoint-268/tokenizer_config.json
Special tokens file saved in person-finetuned/checkpoint-268/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: id, ner_tags, tokens. If id, ner_tags, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this me

TrainOutput(global_step=2680, training_loss=0.036825805769037843, metrics={'train_runtime': 1737.2974, 'train_samples_per_second': 24.653, 'train_steps_per_second': 1.543, 'total_flos': 4568205739822092.0, 'train_loss': 0.036825805769037843, 'epoch': 10.0})

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: id, ner_tags, tokens. If id, ner_tags, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 535
  Batch size = 16


{'epoch': 10.0,
 'eval_accuracy': 0.9771753681392236,
 'eval_f1': 0.8109113453192808,
 'eval_loss': 0.07653242349624634,
 'eval_precision': 0.8084054388133498,
 'eval_recall': 0.8134328358208955,
 'eval_runtime': 7.3233,
 'eval_samples_per_second': 73.054,
 'eval_steps_per_second': 4.643}

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [training_tagset[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [training_tagset[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

In [ ]:
trainer.save_model("person_final")

Saving model checkpoint to person_final
Configuration saved in person_final/config.json
Model weights saved in person_final/pytorch_model.bin
tokenizer config file saved in person_final/tokenizer_config.json
Special tokens file saved in person_final/special_tokens_map.json


In [ ]:
from google.colab import files
import shutil
shutil.make_archive("person_fr", 'zip', "person_final")

'/content/person_fr.zip'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp "/content/person_fr.zip" /content/drive/MyDrive

Mounted at /content/drive


# Predictions

In [ ]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer

language = "fr"

# Change path if needed:
model = AutoModelForTokenClassification.from_pretrained("hidden_vs_public_final")
tokenizer = AutoTokenizer.from_pretrained("hidden_vs_public_final")
tokenizer.add_tokens(["[PERS]"])

In [ ]:
def get_labels(output):
    list_entities = []
    dict_labels = {"LABEL_0": "O",
                   "LABEL_1": "B-PUBLIC",
                   "LABEL_2": "I-PUBLIC",
                   "LABEL_3": "B-HIDDEN",
                   "LABEL_4": "I-HIDDEN",
                  }
    for entity_group in output:
        if entity_group["word"] == "[PERS]":
            list_entities.append(dict_labels[entity_group["entity_group"]])
    return list_entities

In [ ]:
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

In [ ]:
output = ner_pipeline("Le président [PERS] a été invité à la conference de Paris.")
get_labels(output)

In [ ]:
output = ner_pipeline("[PERS] [PERS], menuisier, 37 ans, a été cambriolé.")
get_labels(output)

In [ ]:
sentence = "Une réunion trimestrielle de la corporation de cet arrondissement s'est tenue au Guildhall, lundi, sous la présidence de [PERS] [PERS]."

In [ ]:
output = ner_pipeline(sentence)
get_labels(output)

In [ ]:
sentence = "Une réunion trimestrielle de la corporation de cet arrondissement s'est tenue au Guildhall, lundi, où habite [PERS] [PERS]."

In [ ]:
output = ner_pipeline(sentence)
get_labels(output)

In [ ]:
sentence = "[PERS] [PERS] a été poignardé alors qu'il se rendait à l'opéra."
output = ner_pipeline(sentence)
get_labels(output)

# Pushing to huggingface hub

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
trainer.push_to_hub()

OSError: ignored